In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import igraph as ig
from collections import Counter
import pickle
import math
import random

from src import acgraph as acg
from src import bcloader as bcl
from src import utils as ut
from src import bcgraph as bcg
from efficient_apriori import apriori
from src import xsmining as xsm

### Load access control graph

In [2]:
usrlabels, usrlabel_to_attvals, usrname_to_usrlabel, \
reslabels, reslabel_to_attvals, resname_to_reslabel, usrlabel_to_reslabel = xsm.load_dataset(name='HC')
print('Num entries:', len(usrlabel_to_reslabel))

Num entries: 1576


In [3]:
#---Create access control graph---
gur = xsm.load_gur(usrlabels, reslabels, usrlabel_to_reslabel, save=False)
print('Num of users:', len(usrlabels))
print('Num of resources:', len(reslabels))
print('Num of edges:', len(gur.es))

Num of users: 200
Num of resources: 420
Num of edges: 1576


In [4]:
#Get attribute-values of users
userlog_objs = list(usrlabel_to_attvals.values())

resvidx_to_neighattvals = dict()
for vidx,v in enumerate(gur.vs):
    if v['type'] == True:
        neighattvals = set()
        for usrvidx in gur.neighbors(v):
            neighattvals |= set(userlog_objs[usrvidx])
            
        resvidx_to_neighattvals[vidx] = list(neighattvals)

### Useful functions

In [5]:
def plot_bins(usrvidx_resvidx_tups):
    
    idxs_entry_events = []
    for v in gur.vs:
        if v['type']:
            idx = v.index - len(usrlabels)
            for _ in range(gur.degree(v)):
                idxs_entry_events.append(idx)
    
    
    idxs_example_events = []
    for tup in usrvidx_resvidx_tups:
        resvidx = tup[1]
        idx = resvidx - len(usrlabels)
        idxs_example_events.append(idx)
        
    
    bins = np.linspace(0, len(reslabels), 10)
    plt.hist(idxs_entry_events,bins=bins,color='tab:blue', density=True)
    plt.hist(idxs_example_events,bins=bins,color='tab:red', alpha=0.4, density=True)
    plt.plot()
    
def cov_resources(usrvidx_resvidx_tups):
    residxsset = set()
    for tup in usrvidx_resvidx_tups:
        resvidx = tup[1]
        residxsset.add(resvidx)
        
    return len(residxsset)/len(reslabels)

In [6]:
def next_vertex(gur, v):
    neighvidxs = gur.neighbors(v)
    i = random.randint(0,len(neighvidxs)-1)
    return gur.vs[neighvidxs[i]]
    

def get_entry_tupes(gur, max_hops, alpha, sd):
    random.seed(sd)
    usrvidx_resvidx_tups = list()
    
    for v in gur.vs:
        if v['type']: #Only resources
            neighvidxsset = set(gur.neighbors(v))
            deg = len(neighvidxsset)
            usrvidxsset = set()

            count = 0
            while len(usrvidxsset) < alpha*deg:
                v_ = v
                for i in range(max_hops):
                    v2 = next_vertex(gur, v_)
                    if i == max_hops-1 and not v2.index in neighvidxsset:
                        usrvidxsset.add(v2.index)
                    else:
                        v_ = v2
                count += 1
                
                if count > 100000:
                    break
                    
            for usrvidx in list(usrvidxsset):
                usrvidx_resvidx_tups.append((usrvidx,v.index))
        
    return usrvidx_resvidx_tups


def filter_atts1(gur, usrvidx_resvidx_tups, minth, maxth):
    usrvidx_resvidx_tups_ = []
    
    for tup in usrvidx_resvidx_tups:
        usridx = tup[0]
        usrattvalsset = set(usrlabel_to_attvals[usrlabels[usridx]])
        
        resvidx = tup[1]
        neighattvalsset = set(resvidx_to_neighattvals[resvidx])
        
        x = len(usrattvalsset&neighattvalsset)
        if x > minth and x <= maxth:
            usrvidx_resvidx_tups_.append(tup)
            
    return usrvidx_resvidx_tups_


def filter_atts2(gur, usrvidx_resvidx_tups, minth, maxth):
    usrvidx_resvidx_tups_ = []
    
    for tup in usrvidx_resvidx_tups:
        usridx = tup[0]
        usrattvalsset = set(usrlabel_to_attvals[usrlabels[usridx]])
        
        residx = tup[1] - len(usrlabels) 
        resattvalsset = set(reslabel_to_attvals[reslabels[residx]])
        
        x = len(usrattvalsset&resattvalsset)
        if x > minth and x <= maxth:
            usrvidx_resvidx_tups_.append(tup)
            
    return usrvidx_resvidx_tups_
    

def get_non_entries(gur, numentries, sd):
    edges_tuples = []
    for e in gur.es:
        edges_tuples.append((e.source,e.target))

    usrvidxs = []
    resvidxs = []
    for v in gur.vs:
        if v['type']:
            resvidxs.append(v.index)
        else:
            usrvidxs.append(v.index)

    random.seed(sd)
    usrvidxs_random = random.choices(usrvidxs,k=10*numentries)
    resvidxs_random = random.choices(resvidxs,k=10*numentries)

    i = 0
    j = 0
    usrvidx_resvidx_tups_non = []
    while i < numentries:
        tup = (usrvidxs_random[j], resvidxs_random[j])
        if not tup in edges_tuples: 
            usrvidx_resvidx_tups_non.append(tup)
            i += 1
        j += 1
        
    return usrvidx_resvidx_tups_non


### Create positive synthetic examples

In [26]:
#Through DIST method
usrvidx_resvidx_tups = get_entry_tupes(gur, max_hops=3, alpha=2, sd=43)
usrvidx_resvidx_tups_pos_DIST = filter_atts1(gur, usrvidx_resvidx_tups, minth=0, maxth=10)
usrvidx_resvidx_tups_pos_DIST = filter_atts2(gur, usrvidx_resvidx_tups_pos_DIST, minth=0, maxth=1)
print('Num of pos examples:', len(usrvidx_resvidx_tups_pos_DIST))
print('Covered resources:', cov_resources(usrvidx_resvidx_tups_pos_DIST))

Num of pos examples: 744
Covered resources: 0.5904761904761905


In [27]:
num_pos_examples = 150 #Because HC's ACG has 1.5K edges (80-20)
random.seed(13)
usrvidx_resvidx_tups_pos_DIST_sam = random.sample(usrvidx_resvidx_tups_pos_DIST,num_pos_examples)

### Create negative synthetic examples

In [53]:
#Through DIST method
usrvidx_resvidx_tups = get_entry_tupes(gur, max_hops=5, alpha=2, sd=13)
usrvidx_resvidx_tups_neg_DIST = usrvidx_resvidx_tups
#usrvidx_resvidx_tups_neg_DIST = filter_atts1(gur, usrvidx_resvidx_tups, minth=0, maxth=1)
print('Num of neg examples:', len(usrvidx_resvidx_tups_neg_DIST))
print('Covered resources:', cov_resources(usrvidx_resvidx_tups_neg_DIST))

Num of neg examples: 3152
Covered resources: 1.0


In [54]:
#Through RANDOM method
usrvidx_resvidx_tups = get_non_entries(gur, numentries=5000, sd=13)
usrvidx_resvidx_tups_neg_RANDOM = usrvidx_resvidx_tups
#usrvidx_resvidx_tups_neg_RANDOM = filter_atts1(gur, usrvidx_resvidx_tups, minth=0, maxth=1)
print('Num of neg examples:', len(usrvidx_resvidx_tups_neg_RANDOM))
print('Covered resources:', cov_resources(usrvidx_resvidx_tups_neg_RANDOM))

Num of neg examples: 5000
Covered resources: 1.0


In [55]:
num_neg_examples = 150 #Because HC's ACG has 1.5K edges (80-20)
random.seed(13)
usrvidx_resvidx_tups_neg_DIST_sam = random.sample(usrvidx_resvidx_tups_neg_DIST,num_neg_examples)
random.seed(13)
usrvidx_resvidx_tups_neg_RANDOM_sam = random.sample(usrvidx_resvidx_tups_neg_RANDOM,num_neg_examples)

In [56]:
def print_measures(num_pos, num_neg, num_truepos, num_falsepos):
    print('TPR (Recall):', num_truepos/num_pos)
    #print('FPR', num_falsepos/num_neg)
    recall = num_truepos/num_pos
    precision = num_truepos/(num_truepos+num_falsepos)
    fscore = 2*recall*precision/(recall+precision)
    print('Precision:', precision)
    print('Fscore:', fscore)
    
    num_trueneg = num_neg - num_falsepos
    acc = (num_truepos+num_trueneg)/(num_pos+num_neg)
    print('Accuracy:', acc)

In [57]:
def evaluate_rules(usrvidx_resvidx_tups, rules):
    num_trues = 0
    for tup in usrvidx_resvidx_tups:
        usridx = tup[0]
        residx = tup[1] - len(usrlabels)
        
        usrattvals = usrlabel_to_attvals[usrlabels[usridx]]
        resattvals = reslabel_to_attvals[reslabels[residx]]

        for rule in rules:
            x1 = len(set(usrattvals)&set(rule[0]))
            if x1 == len(rule[0]):
                x2 = len(set(resattvals)&set(rule[1]))
                if x2 == len(rule[1]):
                    num_trues += 1
                    break
                    
    return num_trues


### Evaluate biclique graph patterns

In [17]:
#---Load bicliques---
k = (1,1)
bcs = xsm.load_bicliques('HC', gur, k, usrlabel_to_attvals, reslabel_to_attvals)
print('Num of bcs:', len(bcs))
subbcs = xsm.get_subbcs(bcs)
print('Num of bcs with regular patterns:', len(subbcs))

Num of bcs: 261
Num of bcs with regular patterns: 105


In [18]:
#---Create graph of bicliques---
bcgraph = xsm.load_bcgraph(gur, subbcs, save=False)
print('Num of edges:', len(bcgraph.es))

Num of edges: 382


In [19]:
#---Create and evaluate rules from graph-patterns---
rules,bcs_list = xsm.compute_gprules(bcgraph, subbcs)

rules_ = []
for rule in rules:
    if len(rule[0]) > 0 and len(rule[1]) > 0:
        rules_.append(rule)
        
rules = rules_
print('Num rules:', len(rules))

Num rules: 104


In [58]:
#Pos: dist, Neg: dist
num_truepos = evaluate_rules(usrvidx_resvidx_tups_pos_DIST_sam, rules)
num_falsepos = evaluate_rules(usrvidx_resvidx_tups_neg_DIST_sam, rules)
print_measures(num_pos_examples, num_neg_examples, num_truepos, num_falsepos)

TPR (Recall): 1.0
Precision: 0.6666666666666666
Fscore: 0.8
Accuracy: 0.75


In [59]:
#Pos: dist, Neg: dist
num_truepos = evaluate_rules(usrvidx_resvidx_tups_pos_DIST_sam, rules)
num_falsepos = evaluate_rules(usrvidx_resvidx_tups_neg_RANDOM_sam, rules)
print_measures(num_pos_examples, num_neg_examples, num_truepos, num_falsepos)

TPR (Recall): 1.0
Precision: 0.9375
Fscore: 0.967741935483871
Accuracy: 0.9666666666666667


### Evaluate frequent patterns

In [60]:
#---Create and evaluate rules from avpatterns---
minsup = 0.01
rules_fp = xsm.compute_avprules(minsup, usrlabel_to_attvals, reslabel_to_attvals, usrlabel_to_reslabel)

rules_ = []
for rule in rules_fp:
    if len(rule[0]) > 0 and len(rule[1]) > 0:
        rules_.append(rule)
        
rules_fp = rules_

In [61]:
#Pos: dist, Neg: dist
num_truepos = evaluate_rules(usrvidx_resvidx_tups_pos_DIST_sam, rules_fp)
num_falsepos = evaluate_rules(usrvidx_resvidx_tups_neg_DIST_sam, rules_fp)
print_measures(num_pos_examples, num_neg_examples, num_truepos, num_falsepos)

TPR (Recall): 1.0
Precision: 0.5190311418685121
Fscore: 0.683371298405467
Accuracy: 0.5366666666666666


In [62]:
#Pos: dist, Neg: dist
num_truepos = evaluate_rules(usrvidx_resvidx_tups_pos_DIST_sam, rules_fp)
num_falsepos = evaluate_rules(usrvidx_resvidx_tups_neg_RANDOM_sam, rules_fp)
print_measures(num_pos_examples, num_neg_examples, num_truepos, num_falsepos)

TPR (Recall): 1.0
Precision: 0.6437768240343348
Fscore: 0.7832898172323759
Accuracy: 0.7233333333333334
